# Chi-Square Independence Test

---

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.power import TTestIndPower

plt.style.use("fivethirtyeight")
%matplotlib inline

---

Un altra possibile domanda di ricerca è stabilire se due distinte variabili categoriche (di cui almeno una di esse ha più di due livelli) sono correlate (dipendenti o indipendenti, non si sta parlando di *causalità*). Questo test prende il nome di **chi-square test of independence**.

L'unica differenza con il *chi-square GOF* è il calcolo dei *degrees of freedom* della *chi-square distribution*, che coincidono con il prodotto fra il numero di livelli della prima categorica meno uno e il numero di livelli della seconda categorica meno uno.

$$\large df=(R-1)\times(C-1)$$

La *chi-square statistic* si calcola sempre nello stesso modo, mentre una maggiore attenzione va data nel calcolo degli *expected counts*. Una formula utile è:

$$\large Expected=\frac{Row\;Total\times Column\;Total}{Table\;Total}$$

Si parla di righe e colonne perché, in genere, quando si lavora con due categoriche si tende a riassumere i *count* totali in una *two-way table*. Sulle righe spesso troviamo la categorica a due livelli, mentre sulle colonne quella a tre o più livelli.

Il calcolo degli **expected counts** è composto da più step. Bisogna calcolare, per ogni valore della categorica *in righe*, il *count* totale e dividerlo per il numero totale di soggetti coinvolti. In questa maniera si ottiene il *rate* generale di quel valore della categorica. Tale valore dev'essere poi moltiplicato per tutti gli incroci della categorica *in colonne* per ottenere i veri e propri *expected counts* di ogni cella della tabella.

Ricordiamo, infine, che le condizioni da rispettare per realizzare il test sono identiche al *Chi-Square GOF*.

- **Independence**: le osservazioni raccolte nel sample devono essere indipendenti, quindi dev'essere applicato il *random sampling/assignment* e, nel caso di *sampling without replacement*, il *sample size* dev'essere inferiore al 10% della popolazione. Inoltre, ogni istanza non può appartenere a più di un livello della variabile categorica, cioè può contribuire al *count* totale solo per una di esse.


- **Sample Size**: ogni livello della variabile categorica deve avere almeno 5 *expected cases*.


- **Expected**: Nessuna cella deve presentare *expected count* nullo; non più del 20% delle celle devono presentare *expected count* inferiore a 5.

---

E' stato chiesto a 11160 persone che dichiarano di assumere alcohol se hanno mai praticato il binge drinking (opzioni: frequent, occasional, never) e se hanno mai avuto guai con la polizia (opzioni: yes, no). Vediamo i risultati:

- Trouble/Never = 71
- NoTrouble/Never = 4992
- Trouble/Occasional = 154
- NoTrouble/Occasional = 2808
- Trouble/Frequent = 398
- NoTrouble/Frequent = 2737

L'ipotesi nulla è che il binge drinking e l'avere problemi con la polizia siano variabili indipendenti. L'ipotesi alternativa è che non sono indipendenti.

In [2]:
d = {"Problem": ["Trouble","NoTrouble"],
     "Never": [71, 4992],
     "Occasional": [154, 2808],
     "Frequent": [398, 2737]}

df = pd.DataFrame(d)
df.set_index("Problem", inplace = True)
df

,Never,Occasional,Frequent
Problem,,,
Trouble,71,154,398
NoTrouble,4992,2808,2737


In [3]:
chi2, p_value, dof, expected = stats.chi2_contingency(df)

In [4]:
chi2, p_value, dof

(469.5949136516296, 1.0684645945577432e-102, 2)

In [5]:
expected

array([[ 282.63879928,  165.35179211,  175.0094086 ],
       [4780.36120072, 2796.64820789, 2959.9905914 ]])

Il *p-value* è infinitesimale. Le variabili non sono indipendenti!

---

**[Esempio]** C'è una qualche relazione fra la regione e l'età degli abitanti?

In [6]:
cen = pd.read_csv("../data/small-census.csv")

In [7]:
crosstab = pd.crosstab(cen["region"], cen["agecat"])
crosstab

agecat,19-29,30-34,35+
region,,,
N Cntrl,162,92,30
NE,46,83,37
South,139,68,43
West,160,73,23


In [8]:
chi2, p_value, dof, expected = stats.chi2_contingency(crosstab)

In [9]:
chi2, p_value, dof

(61.28767688406035, 2.463382670201335e-11, 6)

In [10]:
expected

array([[150.61506276,  93.87447699,  39.51046025],
       [ 88.03556485,  54.87029289,  23.09414226],
       [132.58368201,  82.63598326,  34.78033473],
       [135.76569038,  84.61924686,  35.61506276]])

C'è relazione fra le variabili, gli *observed counts* deviano particolarmente dagli *expected counts*.

---